In [1]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('./data/전처리_샘플뉴스.csv')
df

,title,text,url,media,date
0,방역 2.5단계 뒤 을씨년스럽게 변한 '스벅'···별천지된 '파바',서울의 밤이 달라졌다. 수도권을 대상으로 이른바 ‘사회적 거리두기 2.5단계’가 시...,https://news.naver.com/main/read.nhn?mode=LPOD...,중앙일보,2020-08-31
1,"""기적 같은 선방""이라지만...현장은 성한 곳이 없다","“50년 동안 사업하면서 소위 ‘깡통’을 세 번 차고도 재기했는데, 이번 위기는 정...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-03
2,13일까지 장맛비… 사흘간 물폭탄 사망·실종 25명,곳곳 수해 초비상 \r\n\r\n 3일 오전 충남 천안과 아산지역에 호우경보가...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-08-04
3,최악 장마에 태풍까지… 내일까지 최고 300㎜,예측하기 어려운 ‘물폭탄’을 동반한 장마가 유례없이 길어지고 있다. 설상가상으로 9...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-08-10
4,은행 예금금리 사상 첫 0%대,대출금리도 2.72% 사상최저\r\n\r\n\r\n\r\n\t\t\t\t은행 예금에...,https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-08-01


In [3]:
# 🤔 하나의 문서를 어떻게 나눌 것 인지?
# -> 하나의 문서를 사용할수도 혹은 문단으로 분리하여 등등
text_res = []
for i in df['text']:
    text_res.extend(i.split('\n'))

In [4]:
okt = Okt()
nouns = [okt.nouns(i) for i in text_res]
len(nouns)

159

In [5]:
# stop_word
stop_word = ['서울','단계']
last_nouns = []
for i in nouns:
    last_nouns.append([j for j in i if j not in stop_word])

In [6]:
# 빈줄 삭제 - 각 문서에서 명사 추출
last_nouns = [i for i in last_nouns if len(i) != 0]
len(last_nouns)

66

In [7]:
# 단어 집합
word_list = sorted(list(set([j for i in last_nouns for j in i])))
print(len(word_list))

980


In [8]:
count = []
for i in last_nouns:
    count.append([])
    for j in word_list:
        count[-1].append(i.count(j))
tf_df = pd.DataFrame(count,columns=word_list)
tf_df

,가건물,가게,가계,가공,가구,가까이,가동,가량,가빈,가운데,...,횟집,효과,후,후퇴,훈,휴가,휴직,흑자,희망고문,희비
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [9]:
# idf
D = len(tf_df)
df = tf_df.astype('bool').sum()
idf = np.log(D/(df+1))
idf
tfidf_df = tf_df*idf
tfidf_df

,가건물,가게,가계,가공,가구,가까이,가동,가량,가빈,가운데,...,횟집,효과,후,후퇴,훈,휴가,휴직,흑자,희망고문,희비
0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,3.496508,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,2.80336,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,2.80336,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
62,0.0,0.00000,9.273127,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
63,0.0,0.00000,0.000000,0.0,0.0,3.091042,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
64,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,3.496508,0.0,0.0,0.0,0.0,0.0


In [10]:
# 코싸인유사도 - 자료별 상관관계
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_df, tfidf_df)
result_df = pd.DataFrame(cos_sim)
result_df

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,1.000000,0.075149,0.082819,0.131495,0.016376,0.000000,0.0,0.051624,0.065985,0.030829,...,0.000000,0.000000,0.000000,0.000000,0.036521,0.000000,0.018103,0.000000,0.0,0.000000
1,0.075149,1.000000,0.114228,0.167932,0.007265,0.000000,0.0,0.031197,0.032577,0.063714,...,0.000000,0.005333,0.000000,0.000000,0.032076,0.020354,0.015900,0.000000,0.0,0.032486
2,0.082819,0.114228,1.000000,0.234196,0.242333,0.000000,0.0,0.053187,0.054376,0.054175,...,0.000000,0.025484,0.028693,0.009095,0.030936,0.009120,0.010183,0.000000,0.0,0.000000
3,0.131495,0.167932,0.234196,1.000000,0.077604,0.122623,0.0,0.044914,0.049312,0.012476,...,0.000000,0.022018,0.000000,0.006341,0.014324,0.015717,0.011821,0.022552,0.0,0.000000
4,0.016376,0.007265,0.242333,0.077604,1.000000,0.000000,0.0,0.019920,0.023078,0.014614,...,0.000000,0.012206,0.007794,0.000000,0.000000,0.041054,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.000000,0.020354,0.009120,0.015717,0.041054,0.000000,0.0,0.019752,0.000000,0.011792,...,0.140286,0.066968,0.072255,0.289214,0.271798,1.000000,0.158358,0.000000,0.0,0.000000
62,0.018103,0.015900,0.010183,0.011821,0.000000,0.000000,0.0,0.000000,0.007746,0.010485,...,0.222048,0.168064,0.185324,0.366594,0.202936,0.158358,1.000000,0.065696,0.0,0.000000
63,0.000000,0.000000,0.000000,0.022552,0.000000,0.000000,0.0,0.000000,0.010436,0.012594,...,0.068874,0.075971,0.054328,0.030278,0.000000,0.000000,0.065696,1.000000,0.0,0.000000
64,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000


In [11]:
result_df[0].sort_values(ascending=False)

0     1.000000
3     0.131495
2     0.082819
28    0.081104
38    0.075649
        ...   
58    0.000000
61    0.000000
63    0.000000
64    0.000000
65    0.000000
Name: 0, Length: 66, dtype: float64

In [12]:
print(' '.join(last_nouns[0]))
print(' '.join(last_nouns[3]))

밤 수도권 대상 사회 거리 두기 시행 음식점 등 오후 대거 문 때문
오후 식당 최모 씨 친구 약속 때문 강제 이 시간 귀가 고 말 층 규모 대형 술집 시행 다음 달 아예 문 이 술집 사장 오후 영업 건 그냥 술집 문 이야기 라며 포장 배달 누가 술집 술 안주 배달 말 서울역 대합실 내 한식당 등 오후 이전 문 패스트푸드 가게 일부 포장 주문 손님 눈 뿐


In [13]:
# 컬럼별 상관관계
tfidf_df.corr()['코로나'].sort_values(ascending=False)

코로나     1.000000
만큼      0.654926
하객      0.583961
통제      0.563392
품       0.563392
          ...   
금지     -0.138035
집중호우   -0.138035
직원     -0.138035
경기도    -0.152464
기자     -0.179060
Name: 코로나, Length: 980, dtype: float64